In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow.keras as tf

In [2]:
ratings_df = pd.read_csv("book-data/ratings.csv")
books_df = pd.read_csv("book-data/books.csv")

In [3]:
ratings_df.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [4]:
books_df.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [5]:
ratings_df.shape

(981756, 3)

In [6]:
books_df.shape

(10000, 23)

In [7]:
ratings_df.user_id.nunique()

53424

In [8]:
books_df.book_id.nunique()

10000

In [9]:
ratings_df.isna().sum()

book_id    0
user_id    0
rating     0
dtype: int64

In [10]:
books_df.isna().sum()

id                              0
book_id                         0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest = train_test_split(ratings_df, test_size=0.2, random_state=1)
print(f"Shape of train data: {Xtrain.shape}")
print(f"Shape of test data: {Xtest.shape}")

Shape of train data: (785404, 3)
Shape of test data: (196352, 3)


## CREATE A NEURAL NETWORK

In [12]:
nbook_id = ratings_df.book_id.nunique()
nuser_id = ratings_df.user_id.nunique()


#Book input network
input_books = tf.layers.Input(shape=[1])
embed_books = tf.layers.Embedding(nbook_id + 1,10)(input_books)
books_out = tf.layers.Flatten()(embed_books)

#user input network
input_users = tf.layers.Input(shape=[1])
embed_users = tf.layers.Embedding(nuser_id + 1,10)(input_users)
users_out = tf.layers.Flatten()(embed_users)

conc_layer = tf.layers.Concatenate()([books_out, users_out])
x = tf.layers.Dense(512, activation='relu')(conc_layer)
x = tf.layers.Dense(128, activation='relu')(x)
x_out = x = tf.layers.Dense(1, activation='relu')(x)

model = tf.Model([input_books, input_users], x_out)

opt = tf.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=opt, loss='mean_squared_error')

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 10)        100010      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 10)        534250      input_2[0][0]                    
______________________________________________________________________________________________

In [15]:
hist = model.fit([Xtrain.book_id, Xtrain.user_id], Xtrain.rating,batch_size=64, epochs=10, verbose=1)

Train on 785404 samples
Epoch 1/10
785404/785404 [==============================] - 122s 155us/sample - loss: 0.6500
Epoch 2/10
785404/785404 [==============================] - 122s 155us/sample - loss: 0.6359
Epoch 3/10
785404/785404 [==============================] - 123s 156us/sample - loss: 0.6229
Epoch 4/10
785404/785404 [==============================] - 128s 163us/sample - loss: 0.6101
Epoch 5/10
785404/785404 [==============================] - 132s 168us/sample - loss: 0.5995
Epoch 6/10
785404/785404 [==============================] - 131s 167us/sample - loss: 0.5885
Epoch 7/10
785404/785404 [==============================] - 132s 168us/sample - loss: 0.5793
Epoch 8/10
785404/785404 [==============================] - 134s 171us/sample - loss: 0.5704
Epoch 9/10
785404/785404 [==============================] - 134s 171us/sample - loss: 0.5622
Epoch 10/10
785404/785404 [==============================] - 131s 167us/sample - loss: 0.5551


In [16]:
from tensorflow.keras.models import load_model
model.save('model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/assets


In [28]:
# tf.utils.plot_model(model)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


## EXTRACT EMBEDDING LAYERS AND MAPPING FOR VISUALIZATION IN TENSORFLOW EMBEDDING PROJECTOR

In [29]:
# Extract embeddings
book_em = model.get_layer('Embed-Books')
book_em_weights = book_em.get_weights()[0]

In [30]:
book_em_weights.shape

(10001, 5)

In [43]:
books_df_copy = books_df.copy()
books_df_copy = books_df_copy.set_index("book_id")
books_df_copy.head(2)

,id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
book_id,,,,,,,,,,,,,,,,,,,,,,
2767052,1,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
3,2,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...


In [58]:
b_id =list(ratings_df.book_id.unique())
b_id.remove(10000)
dict_map = {}
for i in b_id:
    dict_map[i] = books_df_copy.iloc[i]['title']

In [59]:
out_v = open('vecs.tsv', 'w')
out_m = open('meta.tsv', 'w')

for i in b_id:
    book = dict_map[i]
    embeddings = book_em_weights[i]
    out_m.write(book + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    
out_v.close()
out_m.close()

In [113]:
ratings_df.user_id.unique().min()

1

## MAKING PREDICTIONS

In [119]:
book_arr = np.array(b_id)
user = np.array([100 for i in range(len(b_id))])
user[:5]

array([100, 100, 100, 100, 100])

In [120]:
pred = model.predict([book_arr, user])

In [121]:
pred = pred.reshape(-1)

In [122]:
pred_ids = (-pred).argsort()[0:5]
pred_ids

array([4482, 8945, 7946, 1787, 6919])

In [125]:
books_df.iloc[pred_ids]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
4482,4483,24814,24814,25601,21,836221362,9.780836e+12,Bill Watterson,1996.0,It's a Magical World: A Calvin and Hobbes Coll...,It's a Magical World: A Calvin and Hobbes Coll...,eng,4.75,22351,23429,264,36,57,850,3737,18749,https://images.gr-assets.com/books/1437420710m...,https://images.gr-assets.com/books/1437420710s...
8945,8946,46292,46292,45423,91,9646534783,9.789647e+12,Hafez,1380.0,دیوان‎‎ [Dīvān],The Divan,eng,4.63,2773,6323,286,89,116,384,872,4862,https://images.gr-assets.com/books/1327770153m...,https://images.gr-assets.com/books/1327770153s...
7946,7947,5031805,5031805,40769318,96,1433502410,9.781434e+12,"Anonymous, Lane T. Dennis, Wayne A. Grudem",2002.0,NaN,ESV Study Bible,eng,4.76,8953,10784,262,98,90,367,1152,9077,https://images.gr-assets.com/books/1410151002m...,https://images.gr-assets.com/books/1410151002s...
1787,1788,24813,24813,692719,17,836204387,9.780836e+12,Bill Watterson,1995.0,The Calvin and Hobbes Tenth Anniversary Book,The Calvin and Hobbes Tenth Anniversary Book,eng,4.63,48280,48829,346,346,608,3077,8752,36046,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
6919,6920,24815,24815,25602,19,751500283,9.780752e+12,Bill Watterson,1992.0,The Indispensable Calvin and Hobbes: A Calvin ...,The Indispensable Calvin and Hobbes,eng,4.73,14597,16911,325,19,62,666,2969,13195,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


### Prepare Data for use on application

In [127]:
web_book_data = books_df[["book_id", "title", "image_url", "authors"]]

In [20]:
web_book_data = pd.read_csv("web_book_data.csv")

In [21]:
web_book_data = web_book_data.sort_values('book_id')

In [22]:
web_book_data.head()

,book_id,title,image_url,authors
26,1,Harry Potter and the Half-Blood Prince (Harry ...,https://images.gr-assets.com/books/1361039191m...,"J.K. Rowling, Mary GrandPré"
20,2,Harry Potter and the Order of the Phoenix (Har...,https://images.gr-assets.com/books/1387141547m...,"J.K. Rowling, Mary GrandPré"
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,https://images.gr-assets.com/books/1474154022m...,"J.K. Rowling, Mary GrandPré"
17,5,Harry Potter and the Prisoner of Azkaban (Harr...,https://images.gr-assets.com/books/1499277281m...,"J.K. Rowling, Mary GrandPré, Rufus Beck"
23,6,Harry Potter and the Goblet of Fire (Harry Pot...,https://images.gr-assets.com/books/1361482611m...,"J.K. Rowling, Mary GrandPré"


In [23]:
web_book_data.to_json(r'web_book_data.json', orient='records')